In [1]:
import requests
from pprint import pprint
import pandas as pd
import pymysql
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import uuid
sns.set()
%matplotlib inline

In [2]:
db = pymysql.connect(host='localhost', user='root', database='admin_main', port = 3307, charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)

# Create a cursor object
cursorObject = db.cursor()

feedback_Query = "select * from feedbacks"
feedbackm_Query =  "select * from feedback_meta"
feedbackcat_Query = "select * from feedback_categories"
feedbackch_Query = "select * from feedback_choices"
feedbackq_Query = "select * from feedback_questions"


cursorObject.execute(feedback_Query)
cursorObject.execute(feedbackm_Query)
cursorObject.execute(feedbackcat_Query)
cursorObject.execute(feedbackch_Query)
cursorObject.execute(feedbackq_Query)

feedback_df = pd.read_sql(feedback_Query, db)
feedbackm_df = pd.read_sql(feedbackm_Query, db)
feedbackcat_df = pd.read_sql(feedbackcat_Query, db)
feedbackch_df = pd.read_sql(feedbackch_Query, db)
feedbackq_df = pd.read_sql(feedbackq_Query, db)

In [3]:
feedback_new = feedback_df.drop(feedback_df.columns.difference(['feedback_category_id', 'status', 'session_id','is_handoff', 'agent_id', 'created_at']), 1)
feedbackm_new = feedbackm_df.drop(feedbackm_df.columns.difference(['feedback_id', 'feedback_question_id', 'feedback_choice_id', 'value', 'created_at']), 1)
feedbackcat_new = feedbackcat_df.drop(feedbackcat_df.columns.difference(['id', 'name']), 1)
feedbackch_new = feedbackch_df.drop(feedbackch_df.columns.difference(['id', 'label']), 1)
feedbackq_new = feedbackq_df.drop(feedbackq_df.columns.difference(['id', 'feedback_category_id', 'question_number']), 1)

In [4]:
feedback_new.rename(columns={'feedback_category_id': 'id'}, inplace=True)
feedbackmeta = pd.merge(feedback_new, feedbackcat_new, on='id', how='inner')

In [5]:
feedbackm_new.rename(columns={'feedback_question_id': 'id'}, inplace=True)
feedbackmeta1 = pd.merge(feedbackm_new, feedbackq_new, on='id', how='inner')
feedbackmeta1 = feedbackmeta1.drop(['id'], 1)
feedbackmeta1.rename(columns={'feedback_choice_id': 'id'}, inplace=True)
feedbackmeta2 = feedbackmeta1.merge(feedbackch_new, on='id', how='left')

In [6]:
feedbackmeta.insert(loc=2, column='year_month', value=feedbackmeta['created_at'].map(lambda x: 100*x.year + x.month))
feedbackmeta.insert(loc=3, column='month', value=feedbackmeta.created_at.dt.month)
feedbackmeta.insert(loc=4, column='day', value=feedbackmeta.created_at.dt.dayofweek+1)
feedbackmeta.insert(loc=5, column='hour', value=feedbackmeta.created_at.dt.hour)

def time(row):
    if 4 <= row['hour'] <= 10:
        return 'Morning'
    if 11 <= row['hour'] <= 16:
        return 'Noon'
    if 17 <= row['hour'] <= 21:
        return 'Evening'
    if row['hour'] <= 3 or row['hour'] >= 22:
        return 'Night'
    else:
        return 'No hour'

feedbackmeta['time'] = feedbackmeta.apply(lambda row: time(row), axis=1)
day_map = {1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 4: 'Thursday', 5: 'Friday', 6: 'Saturday', 7: 'Sunday'}
feedbackmeta['the_day'] = feedbackmeta['day'].map(day_map)

feedbackmeta2.insert(loc=2, column='year_month', value=feedbackmeta2['created_at'].map(lambda x: 100*x.year + x.month))
feedbackmeta2.insert(loc=3, column='month', value=feedbackmeta2.created_at.dt.month)
feedbackmeta2.insert(loc=4, column='day', value=feedbackmeta2.created_at.dt.dayofweek+1)
feedbackmeta2.insert(loc=5, column='hour', value=feedbackmeta2.created_at.dt.hour)

def time(row):
    if 4 <= row['hour'] <= 10:
        return 'Morning'
    if 11 <= row['hour'] <= 16:
        return 'Noon'
    if 17 <= row['hour'] <= 21:
        return 'Evening'
    if row['hour'] <= 3 or row['hour'] >= 22:
        return 'Night'
    else:
        return 'No hour'

feedbackmeta2['time'] = feedbackmeta2.apply(lambda row: time(row), axis=1)
day_map = {1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 4: 'Thursday', 5: 'Friday', 6: 'Saturday', 7: 'Sunday'}
feedbackmeta2['the_day'] = feedbackmeta2['day'].map(day_map)


In [ ]:
start_date = "2020-03-01 00:00:00"
end_date = "2020-03-31 23:59:59"
feedbackmeta = feedbackmeta[feedbackmeta.created_at.between(start_date,end_date)]

In [7]:
writer = pd.ExcelWriter("/Users/AF/Downloads/Mcdo data/feedbacks.xlsx", engine='xlsxwriter')

feedbackmeta.to_excel(writer, sheet_name='mcdo feedbacks')
feedbackmeta2.to_excel(writer, sheet_name='feedback meta')

writer.save()